In [1]:
original = open('b01.txt')
encr = open('b01_enc.txt')

 

In [ ]:
og = original.readlines()
og 

In [ ]:
encr_lines = encr.readlines()
encr_lines

In [ ]:
from typing import Dict, List

def group_registers(lines: List[str]) -> Dict[int, List[str]]:
    m : Dict[int, List[str]] = {}
    i = 0
    while i < len(lines):
        line = lines[i]
        if "ID:" in line:
            group_id = line.split(",")[0].split(":")[1]
            # print(group_id)
            i += 3  # Skip the 2 lines after ID
            reg_names = []
            while i < len(lines):
                line = lines[i]
                if "ID:" in line or line == "\n":
                    break
                reg_name = line.split(",")[0]
                reg_names.append(reg_name)
                # print(reg_name)
                i += 1
            m[int(group_id)] = reg_names
            
        i += 1
    
    return m


group_registers(encr_lines)

In [ ]:
group_registers(og)

In [6]:
og_register_group 		= group_registers(og)
encr_register_group 	= group_registers(encr_lines)


In [ ]:


from pprint import pprint


def replace_str_with_num(m_og:Dict[int, List[str]], m_enc:Dict[int, List[str]]):
	m_1 : Dict[int, List[int]] = {}
	m_2 : Dict[int, List[int]] = {}
	
	### Filling str_to_num dictionary
	str_to_num : Dict[str, int] = {}
	for group_id, reg_names in m_og.items():
		for reg_name in reg_names:
			if reg_name not in str_to_num:
				str_to_num[reg_name] = len(str_to_num)
	for group_id, reg_names in m_enc.items():
		for reg_name in reg_names:
			if reg_name not in str_to_num:
				str_to_num[reg_name] = len(str_to_num)
				
	### Filling m_1 and m_2
	for group_id, reg_names in m_og.items():
		m_1[group_id] = [str_to_num[reg_name] for reg_name in reg_names]
		
	for group_id, reg_names in m_enc.items():
		m_2[group_id] = [str_to_num[reg_name] for reg_name in reg_names]
	
	return m_1, m_2	


og_m, encr_m = replace_str_with_num(og_register_group, encr_register_group)
pprint(og_m)
pprint(encr_m)

In [ ]:
from typing import Dict, List, Set
from sklearn.metrics.cluster import normalized_mutual_info_score

def calculate_nmi(old_clusters:Dict[int, List[int]], new_clusters:Dict[int, List[int]]):
    # Flatten the clusters to create label lists
    def flatten_clusters(clusters:Dict[int, List[int]], all_nodes :list):
        labels = [0] * len(all_nodes)  # Initialize with -1 for nodes not in any cluster
        for cluster_id, nodes in clusters.items():
            for node in nodes:
                labels[node] = cluster_id
        return labels

    # Get all unique nodes
    all_nodes: Set[int] = set()
    for cluster in old_clusters.values():
        all_nodes.update(cluster)
    for cluster in new_clusters.values():
        all_nodes.update(cluster)
    all_nodes_ = sorted(all_nodes)

    old_labels = flatten_clusters(old_clusters, all_nodes_)
    new_labels = flatten_clusters(new_clusters, all_nodes_)
    print('old_nodes = ', old_labels)
    print('new_nodes = ', new_labels)
    # Calculate NMI
    nmi_score = normalized_mutual_info_score(old_labels, new_labels)
    print(f"NMI Score: {nmi_score}")
    
    return nmi_score

calculate_nmi(og_m, encr_m)


In [ ]:
import os 
for file in os.listdir('.'):
    if file.endswith('_enc.txt'):
        original =  open( file.replace('_enc.txt', '.txt') ) 
        encr = open(file)
        
        og = original.readlines()
        encr_lines = encr.readlines()
        
        og_register_group 		= group_registers(og)
        encr_register_group 	= group_registers(encr_lines)
        
        og_m, encr_m = replace_str_with_num(og_register_group, encr_register_group)
        
        print(f"Comparing {file} with {file.replace('_enc.txt', '.txt')}")
        calculate_nmi(og_m, encr_m)
        print()
        print()
        

In [ ]:
import os 

list_of_files = ['b07.v', "b09.v",  "b11.v"]
# ./../Tools/hal/build/bin/hal -i b17_encrypted.v -gl ./../Tools/hal/examples/toy_cipher/XILINX_UNISIM.hgl
for file in list_of_files:
    encr = file.replace('.v', '_encrypted.v')
    
    # os.system(f"./../Tools/hal/build/bin/hal -i {file} -gl ./../Tools/hal/examples/toy_cipher/XILINX_UNISIM.hgl")
    os.system(f"./../Tools/hal/build/bin/hal -i {encr} -gl ./../Tools/hal/examples/toy_cipher/XILINX_UNISIM.hgl")

In [93]:

def get_script_for_circuit(a):
	return """from hal_plugins import dataflow

config = dataflow.Configuration(netlist)
config = config.with_flip_flops()

res = dataflow.analyze(config)


config = dataflow.Configuration(netlist)
config = config.with_flip_flops()

res = dataflow.analyze(config)
name = \""""  + a + """\"  ##### Change this name for every circuit
print(f" ----- {name} ------  ")
for group, gates in res.get_groups().items():
    print(f"gates in group {group}   ")
    for gate in gates:
        print(f"{gate.name}")


res.write_txt(f"./dana_testing/{name}.txt")
res.write_dot(f"./dana_testing/{name}.dot")
""" 

In [ ]:
print(get_script_for_circuit('b01'))

In [ ]:
import os 
for file in os.listdir('.'):
	if file.endswith('.dot'):
		print('Converting ', file)
		should_we_convert = input('Do you want to convert this file? (y/n)')
		if should_we_convert == 'y':
			os.system(f"dot -Tpng {file} -o {file.replace('.dot', '.png')}")